# IIR2D Explainer Notebook: Fast Filters, Weird Art, Real Utility

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fyremael/iir2d/blob/main/docs/notebooks/IIR2D_Explainer_Colab.ipynb)

This notebook teaches **what IIR2D is**, **how to use all 8 filters**, and **why it matters** for image/video pipelines.

We'll use playful exemplars (cosmic portrait, mountain scroll, microbe swarm), then map them to production intuition.

## What You'll Do

1. Boot the environment (Colab-friendly).
2. Generate entertaining synthetic scenes.
3. Run filters `1..8` using the canonical IIR2D CPU reference.
4. Compare border modes and precisions.
5. Run a mini temporal/video-style demo.
6. Get a quick benchmark and optional JAX CUDA path check.

In [ ]:
# Colab + local setup
import os
import sys
import subprocess
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    repo = Path("/content/iir2d")
    if not repo.exists():
        subprocess.check_call(["git", "clone", "https://github.com/fyremael/iir2d.git", str(repo)])
    os.chdir(repo)

root = Path.cwd().resolve()
if not (root / "scripts").exists():
    candidates = [root.parent, root.parent.parent, root / "iir2d_op"]
    for cand in candidates:
        if cand.exists() and (cand / "scripts").exists():
            root = cand
            break

os.chdir(root)
if str(root) not in sys.path:
    sys.path.insert(0, str(root))
if str(root / "python") not in sys.path:
    sys.path.insert(0, str(root / "python"))

for pkg in ["numpy", "matplotlib"]:
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print(f"ROOT={root}")

In [ ]:
import math
import time
import numpy as np
import matplotlib.pyplot as plt

from scripts.iir2d_cpu_reference import iir2d_cpu_reference

plt.style.use("dark_background")
np.random.seed(42)

FILTER_LABELS = {
    1: "F1 EMA",
    2: "F2 SOS",
    3: "F3 Biquad",
    4: "F4 SOS",
    5: "F5 FB First",
    6: "F6 Deriche-ish",
    7: "F7 Sharper EMA",
    8: "F8 State",
}

def normalize01(x):
    x = np.asarray(x, dtype=np.float32)
    lo, hi = float(x.min()), float(x.max())
    return (x - lo) / (hi - lo + 1e-8)

def show(img, title="", ax=None):
    if ax is None:
        _, ax = plt.subplots(figsize=(6, 4))
    ax.imshow(np.clip(img, 0.0, 1.0))
    ax.set_title(title)
    ax.axis("off")

def apply_iir2d_rgb(img, filter_id=4, border_mode="mirror", precision="f32", border_const=0.0):
    out_dtype = np.float64 if precision == "f64" else np.float32
    out = np.empty(img.shape, dtype=out_dtype)
    for c in range(3):
        out[..., c] = iir2d_cpu_reference(
            img[..., c],
            filter_id=filter_id,
            border_mode=border_mode,
            border_const=float(border_const),
            precision=precision,
        )
    return np.clip(out, 0.0, 1.0)

def make_cosmic_portrait(h=512, w=512):
    y, x = np.mgrid[-1:1:complex(0, h), -1:1:complex(0, w)]
    radial = np.sqrt(x * x + y * y)
    swirl = np.sin(8 * radial - 3 * np.arctan2(y, x))

    bg_r = normalize01(0.2 + 0.1 * np.cos(3 * x) + 0.2 * swirl)
    bg_g = normalize01(0.1 + 0.2 * np.sin(2 * y + 3 * x) + 0.15 * swirl)
    bg_b = normalize01(0.35 + 0.4 * np.cos(2 * radial) + 0.25 * swirl)
    bg = np.stack([bg_r, bg_g, bg_b], axis=-1)

    head = np.exp(-((x / 0.52) ** 2 + ((y + 0.02) / 0.72) ** 2) * 2.5)
    beard = np.exp(-((x / 0.45) ** 2 + ((y - 0.43) / 0.30) ** 2) * 5.0)
    hair = np.exp(-((x / 0.70) ** 2 + ((y + 0.25) / 0.55) ** 2) * 2.6) * (0.6 + 0.4 * np.sin(18 * x + 8 * y))
    eye_l = np.exp(-(((x + 0.18) / 0.07) ** 2 + ((y + 0.06) / 0.05) ** 2) * 8)
    eye_r = np.exp(-(((x - 0.18) / 0.07) ** 2 + ((y + 0.06) / 0.05) ** 2) * 8)

    skin = np.stack([0.95 * head, 0.78 * head, 0.76 * head], axis=-1)
    beard_rgb = np.stack([0.25 * beard, 0.28 * beard, 0.22 * beard], axis=-1)
    hair_rgb = np.stack([0.45 * hair, 0.30 * hair, 0.18 * hair], axis=-1)
    eyes = np.stack([0.45 * (eye_l + eye_r), 0.25 * (eye_l + eye_r), 0.70 * (eye_l + eye_r)], axis=-1)

    return np.clip(0.55 * bg + skin + beard_rgb + hair_rgb + eyes, 0.0, 1.0)

def make_mountain_scroll(h=384, w=640):
    y, x = np.mgrid[0:1:complex(0, h), 0:1:complex(0, w)]
    ridge = 0.45 * np.sin(10 * x + 4 * np.sin(5 * x)) + 0.25 * np.sin(24 * x + 6 * y)
    clouds = 0.4 * np.cos(7 * y + 3 * np.sin(8 * x))
    texture = 0.15 * np.sin(80 * x * y) + 0.12 * np.cos(60 * (x - y))
    base = normalize01(ridge + clouds + texture)

    r = normalize01(base * 0.9 + 0.2 * np.sin(8 * y))
    g = normalize01(base * 0.8 + 0.25 * np.cos(10 * x))
    b = normalize01(base * 1.05 + 0.3 * np.cos(6 * y))

    img = np.stack([r, g, b], axis=-1)

    # Tiny "castle" silhouettes for fun.
    for cx in [0.28, 0.5, 0.72]:
        mask = (np.abs(x - cx) < 0.012) & (y > 0.38) & (y < 0.70)
        img[mask] *= np.array([0.25, 0.23, 0.22])
        roof = (np.abs(x - cx) < 0.02) & (y > 0.34) & (y < 0.40) & (np.abs(x - cx) < (0.02 - (y - 0.34) * 0.3))
        img[roof] *= np.array([0.35, 0.2, 0.2])

    return np.clip(img, 0.0, 1.0)

def make_microbe_swarm(h=512, w=512, n=180):
    rng = np.random.default_rng(7)
    y, x = np.mgrid[0:h, 0:w]
    x = x.astype(np.float32)
    y = y.astype(np.float32)
    img = np.zeros((h, w, 3), dtype=np.float32)

    # Soft radial background
    xn = (x / w) * 2 - 1
    yn = (y / h) * 2 - 1
    rad = np.sqrt(xn * xn + yn * yn)
    img[..., 1] = normalize01(np.exp(-(rad ** 2) * 1.8) * 0.8)
    img[..., 2] = normalize01(np.exp(-(rad ** 2) * 2.6) * 0.5)

    for _ in range(n):
        cx = rng.uniform(0, w)
        cy = rng.uniform(0, h)
        rx = rng.uniform(5, 16)
        ry = rng.uniform(5, 16)
        blob = np.exp(-(((x - cx) / rx) ** 2 + ((y - cy) / ry) ** 2))
        color = np.array([
            rng.uniform(0.3, 1.0),
            rng.uniform(0.2, 0.95),
            rng.uniform(0.1, 0.7),
        ], dtype=np.float32)
        img += blob[..., None] * color[None, None, :] * 0.42

    # Sprinkle checker-ish chips.
    chips = (((x // 32 + y // 32) % 2) == 0).astype(np.float32)
    img += np.stack([0.1 * chips, 0.07 * chips, 0.02 * chips], axis=-1)

    return np.clip(normalize01(img), 0.0, 1.0)

In [ ]:
scenes = {
    "Cosmic Portrait": make_cosmic_portrait(),
    "Mountain Scroll": make_mountain_scroll(),
    "Microbe Swarm": make_microbe_swarm(),
}

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
for ax, (name, img) in zip(axes, scenes.items()):
    show(img, name, ax=ax)
plt.tight_layout()

## All 8 Filters Side-by-Side

This is the fastest way to build intuition: one source, all filter personalities.

In [ ]:
scene_name = "Cosmic Portrait"
src_img = scenes[scene_name]

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
show(src_img, "Original", ax=axes[0, 0])

for idx, fid in enumerate(range(1, 9), start=1):
    r, c = divmod(idx, 3)
    out = apply_iir2d_rgb(src_img, filter_id=fid, border_mode="mirror", precision="f32")
    show(out, f"{FILTER_LABELS[fid]}", ax=axes[r, c])

plt.suptitle(f"IIR2D Filter Character Study — {scene_name}", fontsize=16)
plt.tight_layout()

In [ ]:
src_img = scenes["Mountain Scroll"]
border_modes = ["clamp", "mirror", "wrap", "constant"]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for ax, border in zip(axes, border_modes):
    out = apply_iir2d_rgb(src_img, filter_id=4, border_mode=border, precision="f32", border_const=0.08)
    show(out, f"Border: {border}", ax=ax)

plt.suptitle("Border Semantics Matter (Filter 4)", fontsize=15)
plt.tight_layout()

In [ ]:
src_img = scenes["Microbe Swarm"]

out_f32 = apply_iir2d_rgb(src_img, filter_id=8, border_mode="mirror", precision="f32")
out_mixed = apply_iir2d_rgb(src_img, filter_id=8, border_mode="mirror", precision="mixed")
out_f64 = apply_iir2d_rgb(src_img.astype(np.float64), filter_id=8, border_mode="mirror", precision="f64")

delta_mixed = np.abs(out_f32 - out_mixed).mean()
delta_f64 = np.abs(out_f32 - out_f64.astype(np.float32)).mean()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
show(out_f32, "f32", ax=axes[0])
show(out_mixed, f"mixed (mean |Δ|={delta_mixed:.6f})", ax=axes[1])
show(out_f64, f"f64 (mean |Δ|={delta_f64:.6f})", ax=axes[2])
plt.suptitle("Precision Modes: Visual + Numeric Drift", fontsize=15)
plt.tight_layout()

In [ ]:
def make_videoish_sequence(base, n=12):
    rng = np.random.default_rng(13)
    frames = []
    for t in range(n):
        dx = int(5 * math.sin(2 * math.pi * t / n))
        dy = int(4 * math.cos(2 * math.pi * t / n))
        shifted = np.roll(np.roll(base, dy, axis=0), dx, axis=1)
        noise = rng.normal(0.0, 0.03, size=base.shape).astype(np.float32)
        frame = np.clip(0.9 * shifted + noise, 0.0, 1.0)
        frames.append(frame)
    return frames

def temporal_energy(frames):
    diffs = [np.mean(np.abs(frames[i + 1] - frames[i])) for i in range(len(frames) - 1)]
    return float(np.mean(diffs))

seq_in = make_videoish_sequence(scenes["Cosmic Portrait"], n=12)
seq_out = [apply_iir2d_rgb(f, filter_id=4, border_mode="mirror", precision="f32") for f in seq_in]

e_in = temporal_energy(seq_in)
e_out = temporal_energy(seq_out)

fig, axes = plt.subplots(2, 6, figsize=(22, 8))
for i in range(6):
    show(seq_in[i], f"In t={i}", ax=axes[0, i])
    show(seq_out[i], f"Out t={i}", ax=axes[1, i])
plt.suptitle(
    f"Video-ish sequence (top=raw, bottom=filtered) | mean temporal energy: {e_in:.4f} -> {e_out:.4f}",
    fontsize=14,
)
plt.tight_layout()

In [ ]:
x = np.random.default_rng(0).random((256, 256, 3), dtype=np.float32)

results = []
for fid in range(1, 9):
    t0 = time.perf_counter()
    for _ in range(3):
        _ = apply_iir2d_rgb(x, filter_id=fid, border_mode="mirror", precision="f32")
    dt = (time.perf_counter() - t0) / 3.0
    results.append((fid, dt * 1000.0))

print("CPU reference timing (256x256x3, avg of 3 runs):")
for fid, ms in results:
    print(f"  Filter {fid}: {ms:8.2f} ms")

In [ ]:
# Optional: check whether the JAX custom-call path is available in this runtime.
try:
    import jax
    import jax.numpy as jnp
    from iir2d_jax import iir2d

    x = jnp.asarray(scenes["Cosmic Portrait"][..., 0], dtype=jnp.float32)
    y = iir2d(x, filter_id=4, border="mirror", precision="f32")
    y_np = np.array(y)
    print("JAX devices:", jax.devices())
    print("JAX call ok. Output stats:", float(y_np.min()), float(y_np.max()), float(y_np.mean()))
except Exception as exc:
    print("JAX custom-call path not available in this runtime (expected on many Colab sessions).")
    print("Reason:", exc)

## Takeaways

- IIR2D is a compact primitive with very different personalities across filters `1..8`.
- Border semantics are not cosmetic; they materially change output behavior.
- For video, temporal smoothness effects are easy to observe frame-to-frame.
- Production path: use CUDA bundles (Linux/Windows) for throughput; use CPU reference for parity and contract checks.

If you want, the next iteration can add:
1. live UI sliders (`ipywidgets`),
2. on-the-fly video decode/encode demo,
3. benchmark table export (`csv` + markdown packet).